## 3.1 Data transferred and ingested in butler and access to data

Get the same ras as we did on the summit

In [90]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry
import sqlalchemy

repo = '/repo/embargo'
# raw all colleciton contains all images 
collections=['LATISS/raw/all']
butler = Butler(repo)
cols = butler.registry.queryCollections("*LATISS*raw*")
for c in cols :
    print(c)


butler = Butler(repo, collections=collections)
registry = butler.registry



LATISS/raw/all


In [91]:
dataId={'day_obs':20230509}

dt='raw'
where="exposure.day_obs = 20230509"
datasetRefs = list(registry.queryDatasets(datasetType=dt,where=where))

print(f"There are {len(datasetRefs)} {dt} in collection: {collections} in: {repo} for {where}")

count = 0
for ref in datasetRefs[0], datasetRefs[-1]:
    count = count + 1
    print( ref.dataId['detector'], ref.dataId['exposure'])



There are 572 raw in collection: ['LATISS/raw/all'] in: /repo/embargo for exposure.day_obs = 20230509
0 2023050900001
0 2023050900572


### latency ..
"Measure the delays between completion of readout for each image and the ingestion time recorded in the Butler Registry."
The end of readou is the exposure.timespan.end.
Will have to lookup the ingest time usinf a sql query sine is not exposed directly in butler.

In [92]:
id = datasetRefs[0].id
print (id)
q_string = f"SELECT * FROM oga.dataset WHERE id='{id}'"
with registry._db.query(sqlalchemy.text(q_string)) as q:
    rec = q.all()
    print(rec)



c7f22148-73dd-5a4c-9d55-4a644a1f93eb
[(UUID('c7f22148-73dd-5a4c-9d55-4a644a1f93eb'), 1, datetime.datetime(2023, 5, 9, 19, 37, 11, 321657), 1)]


In [100]:
from astropy import time

res = butler.registry.queryDimensionRecords('exposure', where=where)
print (f"Got Metadata for {res.count()} exposures")
times = []
for count,r in enumerate(res):
    dr = datasetRefs[count]
    id = dr.id
    
    if r.id != dr.dataId['exposure']:
        print(f" The sets to not match up {r.id} != {dr.dataId['exposure']}")
        break
    q_string = f"SELECT ingest_date FROM oga.dataset WHERE id='{id}'"
    with registry._db.query(sqlalchemy.text(q_string)) as q:
        intime = time.Time(q.all()[0][0])
        
    diff = (intime - r.timespan.end ).to('minute')
    #print(r.id,r.timespan.end, intime, diff )
    times.append(diff.value)
    
import numpy as np

print (f"Time between shutter close and ingest mean:{np.mean(times)}, std:{np.std(times)}, var:{np.var(times)} minutes")

Got Metadata for 572 exposures
Time between shutter close and ingest mean:3.979185934702819, std:0.8598480922734872, var:0.7393387417863555 minutes
